In [ ]:
import os
import numpy as np 
import pandas as pd
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
nltk.download('stopwords')

In [ ]:
def TextTreatment(text):
    
    text = RemoveLowerCase(text)
    text = RemovePonctuation(text)
    text = RemoveUnderlines(text)
    text = RemoveStopWords(text)
    text = Lemmatize(text, FrenchStemmer())
    
    # Things to consider rare words removal 
    
    return text

In [ ]:
def RemoveLowerCase(text):
    return ' '.join([x.lower() for x in text.split()])

In [ ]:
def RemovePonctuation (text):
    return re.sub('\W+',' ', text)

In [ ]:
def RemoveUnderlines (text):
    return re.sub('_','', text)

In [ ]:
def RemoveStopWords(text):
    stop = stopwords.words('french', 'english')
    return ' '.join([x for x in text.split(' ') if x not in stop])

In [ ]:
def Lemmatize(text, stemmer):
    return ' '.join([stemmer.stem(w) for w in text.split(' ')])

In [ ]:
path = '../data/raw/node_information/text'
no_nodes = 33226

files = []
for i in range(no_nodes):
    files.append(os.path.join(path, str(i) + '.txt'))

In [ ]:
documents = []

for i in range(len(files)):
    
    if i>0 and i%1000 == 0:
        print('Reading file number ' + str(i))
    
    f = open(files[i], "r", errors = 'ignore')
    text = f.read()
    text = TextTreatment(text)
    documents.append(text)

# Doc2Vec Model Train

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(doc, [i]) for i, doc in enumerate(documents)]

In [ ]:
max_epochs = 50
vec_size = 100
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                workers=12)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch+1))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("../results/models/doc2vec/vec10.model")
print("Model Saved")

# TF-IDF 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
import scipy

In [ ]:
## Create Vocabulary
vocabulary = set()
for i, doc in enumerate(documents):
    vocabulary.update(doc.split(' '))
    
    if i>0 and i%1000 == 0:
        print('Reading document number ' + str(i))
    
    
vocabulary = list(vocabulary)

In [ ]:
len(vocabulary)

In [ ]:
# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary)

# Fit the TfIdf model
tfidf.fit(documents)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(documents)

In [ ]:
#Save sparse matrix for future use
scipy.sparse.save_npz("../results/models/tf-idf_matrix", tfidf_tran)